In [1]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import torch.hub
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset

# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [2]:
myseed = 3231  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)
# "cuda" only when GPUs are available.
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [3]:
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.RandomAffine(degrees=18, translate=(0.15, 0.15), scale=(0.8, 1.2)),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(),
    transforms.ToTensor(),
])

In [4]:
def split_set(path, ratio = 0.8, files = None, shuffle = True):
    
    files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
    if shuffle == True:
        random.shuffle(files)
    if ratio < 1:
        train_image_paths, valid_image_paths = files[:int(ratio*len(files))], files[int(ratio*len(files)):]    
        return train_image_paths, valid_image_paths
    elif ratio == 1:
        test_image_paths = files[:]
        return test_image_paths

In [5]:
class FoodDataset(Dataset):

    def __init__(self,pathes,tfm=test_tfm):
        super(FoodDataset).__init__()
        self.pathes = sorted(pathes)
        print(f"dataset with {len(self.pathes)} images")
        self.transform = tfm
  
    def __len__(self):
        return len(self.pathes)
  
    def __getitem__(self,idx):
        fname = self.pathes[idx]
        im = Image.open(fname)
        im = self.transform(im)
        #im = self.data[idx]
        try:
            label = int(fname.split("\\")[-1].split("_")[0])
        except:
            label = -1 # test has no label
        return im,label

In [6]:
def mixup_data(x, y, alpha=1.0):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [7]:
RestNeXt = torchvision.models.resnext50_32x4d(pretrained=False, num_classes = 11).to(device)
se_resnet50 = torch.hub.load(
    'moskomule/senet.pytorch',
    'se_resnet50',
    pretrained=False, num_classes = 11).to(device)

Using cache found in C:\Users\User/.cache\torch\hub\moskomule_senet.pytorch_master


In [63]:
# import torch.utils.data as data_utils
# indices = torch.arange(16000)
batch_size = 128
_dataset_dir = "./food11"
# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
# train_set = FoodDataset(os.path.join(_dataset_dir,"training"), tfm=train_tfm)
# valid_set = FoodDataset(os.path.join(_dataset_dir,"validation"), tfm=test_tfm)
train_image_paths, valid_image_paths = split_set(os.path.join(_dataset_dir,"all"), ratio = 0.9, shuffle = True)


train_set = FoodDataset(train_image_paths,tfm=train_tfm)
train_set2 = FoodDataset(train_image_paths,tfm=test_tfm)
# train_set3 = FoodDataset(train_image_paths,tfm=train_tfm,mixup=True)
train_set_all = ConcatDataset([train_set,train_set2])
# train_set_all = data_utils.Subset(train_set_all, indices)
valid_set = FoodDataset(valid_image_paths,tfm=test_tfm)

print('training set:',len(train_set_all))

train_loader = DataLoader(train_set_all, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)


dataset with 11966 images
dataset with 11966 images
dataset with 1330 images
training set: 23932


# Eval ResNeXt

In [65]:
# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0
RestNeXt.load_state_dict(torch.load("./RestNeXt_best.ckpt"))
# del valid_pred_all_RestNeXt,train_pred_all_RestNeXt,train_label_all,valid_label_all
# ---------- Training ----------
# Make sure the model is in train mode before training.
RestNeXt.eval()

# These are used to record information in training.


for batch in tqdm(train_loader):

    # A batch consists of image data and corresponding labels.
    imgs, labels = batch
    imgs = imgs.to(device)

    with torch.no_grad():
        train_pred = RestNeXt(imgs.to(device))

    try:
        train_pred_all_RestNeXt = np.vstack([train_pred_all_RestNeXt,train_pred.cpu().data.numpy()])
        train_label_all = np.hstack([train_label_all,labels.numpy()])
    except:
        train_pred_all_RestNeXt = train_pred.cpu().data.numpy()
        train_label_all = labels.numpy()
        

  0%|          | 0/187 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [86]:
RestNeXt.eval()
# Iterate the validation set by batches.
for batch in tqdm(valid_loader):

    # A batch consists of image data and corresponding labels.
    imgs, labels = batch
    #imgs = imgs.half()

    # We don't need gradient in validation.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        valid_pred = RestNeXt(imgs.to(device))
    try:
        valid_pred_all_RestNeXt = np.vstack([valid_pred_all_RestNeXt,valid_pred.cpu().data.numpy()])
        valid_label_all = np.hstack([valid_label_all,labels.numpy()])
    except:
        valid_pred_all_RestNeXt = valid_pred.cpu().data.numpy()
        valid_label_all = labels.numpy()

  0%|          | 0/11 [00:00<?, ?it/s]

# Eval se_resnet

In [66]:
# Initialize trackers, these are not parameters and should not be changed
se_resnet50.load_state_dict(torch.load("./se_resnet50_best.ckpt"))
# ---------- Training ----------
# Make sure the model is in train mode before training.
se_resnet50.eval()
# These are used to record information in training.
for batch in tqdm(train_loader):
    # A batch consists of image data and corresponding labels.
    imgs, labels = batch
    imgs, labels = imgs.to(device), labels.to(device)
    with torch.no_grad():
        train_pred = se_resnet50(imgs.to(device))
    try:
        train_pred_all_se_resnet50 = np.vstack([train_pred_all_se_resnet50,train_pred.cpu().data.numpy()])
    except:
        train_pred_all_se_resnet50 = train_pred.cpu().data.numpy()

RestNeXt.eval()

# Iterate the validation set by batches.
for batch in tqdm(valid_loader):

    # A batch consists of image data and corresponding labels.
    imgs, labels = batch
    #imgs = imgs.half()

    # We don't need gradient in validation.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        valid_pred = se_resnet50(imgs.to(device))
    try:
        valid_pred_all_se_resnet50 = np.vstack([valid_pred_all_se_resnet50,valid_pred.cpu().data.numpy()])
    except:
        valid_pred_all_se_resnet50 = valid_pred.cpu().data.numpy()

  0%|          | 0/187 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

In [71]:
for batch in tqdm(valid_loader):

    # A batch consists of image data and corresponding labels.
    imgs, labels = batch
    #imgs = imgs.half()

    # We don't need gradient in validation.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        valid_pred = se_resnet50(imgs.to(device))
    try:
        valid_pred_all_se_resnet50 = np.vstack([valid_pred_all_se_resnet50,valid_pred.cpu().data.numpy()])
    except:
        valid_pred_all_se_resnet50 = valid_pred.cpu().data.numpy()

  0%|          | 0/11 [00:00<?, ?it/s]

# Train

In [111]:
class PredDataset(Dataset):

    def __init__(self,pred1,pred2,labels):
        super(PredDataset).__init__()
        self.pred1 = pred1
        self.pred2 = pred2
        self.labels = labels
    def __len__(self):
        return len(self.pred1)  
    def __getitem__(self,idx):
        pred = np.hstack([self.pred1[idx],self.pred2[idx]])

        if self.labels != None:
            label = self.labels[idx]
        elif self.labels == None:
            label = -1 # test has no label
        return pred, label

In [98]:
batch_size = 256

train_set2 = PredDataset(train_pred_all_RestNeXt,train_pred_all_se_resnet50,train_label_all)
train_loader2 = DataLoader(train_set2, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
valid_set2 = PredDataset(valid_pred_all_RestNeXt,valid_pred_all_se_resnet50,valid_label_all)
valid_loader2 = DataLoader(valid_set2, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)


In [40]:
class Classifier3(nn.Module):
    def __init__(self, input_dim=22, output_dim=11):
        super(Classifier3, self).__init__()

        self.fc = nn.Sequential(
            nn.Linear(input_dim, output_dim),
        )

    def forward(self, x):
        x = self.fc(x)
        return x

ensemble_model = Classifier3().to(device)

In [99]:
# The number of training epochs and patience.
n_epochs = 30
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(ensemble_model.parameters(), betas=(0.9, 0.98), lr=learning_rate, weight_decay=0.0005)
# optimizer = torch.optim.SGD(RestNeXt.parameters(), lr=learning_rate, momentum=0.9)

In [100]:
# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0

for epoch in range(n_epochs):
    ensemble_model.train()
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader2):

        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        logits = ensemble_model(inputs)
        
        loss = criterion(logits, labels)
        optimizer.zero_grad()

        loss.backward()
        optimizer.step()
        
        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    
    ensemble_model.eval()
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader2):
        inputs, labels = batch
        with torch.no_grad():
            logits = ensemble_model(inputs.to(device))
        loss = criterion(logits, labels.to(device))
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
        valid_loss.append(loss.item())
        valid_accs.append(acc)
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

    # save models
    if valid_acc > best_acc:
        print(f"Best model found at epoch {epoch}, saving model")
        torch.save(ensemble_model.state_dict(),"./ensemble_model_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0

  0%|          | 0/94 [00:00<?, ?it/s]

[ Train | 001/030 ] loss = 0.07689, acc = 0.97719


  0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 001/030 ] loss = 0.02324, acc = 0.98893
Best model found at epoch 0, saving model


  0%|          | 0/94 [00:00<?, ?it/s]

...

[ Valid | 029/030 ] loss = 0.02280, acc = 0.99023
23


  0%|          | 0/94 [00:00<?, ?it/s]

[ Train | 030/030 ] loss = 0.06771, acc = 0.97910


  0%|          | 0/6 [00:00<?, ?it/s]

[ Valid | 030/030 ] loss = 0.02281, acc = 0.99023
24


# TEST

In [101]:
test_image_paths = split_set(os.path.join(_dataset_dir,"test"), ratio = 1, shuffle = False)
test_set = FoodDataset(test_image_paths,tfm=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

test_set1 = FoodDataset(test_image_paths,tfm=train_tfm)
test_set2 = FoodDataset(test_image_paths,tfm=train_tfm)
test_set3 = FoodDataset(test_image_paths,tfm=train_tfm)
test_set4 = FoodDataset(test_image_paths,tfm=train_tfm)
test_set5 = FoodDataset(test_image_paths,tfm=train_tfm)
test_set_all = ConcatDataset([test_set1,test_set2,test_set3,test_set4,test_set5])
test_loader_all = DataLoader(test_set_all, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

dataset with 3347 images
dataset with 3347 images
dataset with 3347 images
dataset with 3347 images
dataset with 3347 images
dataset with 3347 images


In [106]:
# model_best = Classifier().to(device)
RestNeXt.load_state_dict(torch.load("./RestNeXt_best.ckpt"))

RestNeXt.eval()

with torch.no_grad():
    for data,_ in tqdm(test_loader_all):
        test_pred = RestNeXt(data.to(device))
        try:
            test_pred_all = np.vstack([test_pred_all,test_pred.cpu().data.numpy()])
        except:
            test_pred_all = test_pred.cpu().data.numpy()
        
test_pred_all = test_pred_all.reshape(5,3347,11)
test_pred_all = np.mean(test_pred_all,axis=0) 

with torch.no_grad():
    for data,_ in tqdm(test_loader):
        test_pred = RestNeXt(data.to(device))
        try:
            test_pred_result_RestNeXt = np.vstack([test_pred_result,test_pred.cpu().data.numpy()])
        except:
            test_pred_result_RestNeXt = test_pred.cpu().data.numpy()
            
test_pred_final_RestNeXt = (test_pred_result_RestNeXt+test_pred_all)/2

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

In [107]:
# model_best = Classifier().to(device)
se_resnet50.load_state_dict(torch.load(f"se_resnet50_best.ckpt"))

se_resnet50.eval()

with torch.no_grad():
    for data,_ in tqdm(test_loader_all):
        test_pred = se_resnet50(data.to(device))
        try:
            test_pred_all = np.vstack([test_pred_all,test_pred.cpu().data.numpy()])
        except:
            test_pred_all = test_pred.cpu().data.numpy()
        
test_pred_all = test_pred_all.reshape(5,3347,11)
test_pred_all = np.mean(test_pred_all,axis=0) 

with torch.no_grad():
    for data,_ in tqdm(test_loader):
        test_pred = se_resnet50(data.to(device))
        try:
            test_pred_result_se_resnet50 = np.vstack([test_pred_result,test_pred.cpu().data.numpy()])
        except:
            test_pred_result_se_resnet50 = test_pred.cpu().data.numpy()
            
test_pred_final_se_resnet50 = (test_pred_result_se_resnet50+test_pred_all)/2

  0%|          | 0/66 [00:00<?, ?it/s]

  0%|          | 0/14 [00:00<?, ?it/s]

In [112]:
test_set = PredDataset(test_pred_final_RestNeXt,test_pred_final_se_resnet50,None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

In [114]:
# del test_pred_result
ensemble_model = Classifier3().to(device)
ensemble_model.load_state_dict(torch.load(f"ensemble_model_best.ckpt"))
ensemble_model.eval()
prediction = []
with torch.no_grad():
    for data,_ in tqdm(test_loader):
        test_pred = ensemble_model(data.to(device))
        try:
            test_pred_result = np.vstack([test_pred_result,test_pred.cpu().data.numpy()])
        except:
            test_pred_result = test_pred.cpu().data.numpy()        
test_label = np.argmax(test_pred_result, axis=1)
prediction = test_label.squeeze().tolist()

  0%|          | 0/14 [00:00<?, ?it/s]

In [144]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(1,len(test_set)+1)]
df["Category"] = prediction
df.to_csv("ensemblewithTTA.csv",index = False)